# A getting started tutorial of ModelCI installation, Register and convert Model

This is a getting started tutorial for those who are new to ML-ModelCI, including the following steps:

- Setting up environment
- Starting ModelCI service.
- Loading a pretrained resnet50 model from torchvision.
- Register this model into modelHub.

  The converter will automatically convert this PyTorch format model into 2 model : a Open Neural Network eXchange (ONNX) format model and a TorchScipt format model

- Converting the ONNX format model into TensorRT format manually.
- Stopping ModelCI service.

## 1. Installation

Make sure [conda]() and [docker]() are installed before this tutorial, and docker service has already started.

Besides, you should create modelci conda environment based on yml file 

```shell
# create conda environment based on given yml file
wget -q https://raw.githubusercontent.com/cap-ntu/ML-Model-CI/master/environment.yml
conda env create -f environment.yml
```

And starting this Jupyter Notebook in modelci conda environment.

Then we can install the ModelCI python package based on <https://github.com/cap-ntu/ML-Model-CI#using-pip>.

In [1]:
%%bash
# need to install requests package first
pip install setuptools requests==2.23.0
# then install modelci
pip install git+https://github.com/cap-ntu/ML-Model-CI.git@master --use-feature=2020-resolver

3/envs/modelci/lib/python3.7/site-packages (from modelci==1.0.0) (1.18.5)
  Created wheel for modelci: filename=modelci-1.0.0-py3-none-any.whl size=110338 sha256=410c87c981cf58af5bc5ace03da8368d541f0d2a60e5e462602c5819034a418d
  Stored in directory: /tmp/pip-ephem-wheel-cache-1tg4zlhc/wheels/be/22/72/e2eb8d0746bb54bfb75f9dd82721af505f0596f85932785f5b
Successfully built modelci


## 2. Start the ModelCI service

Firstly, we should set some environmemnt variables, especially mongodb related variables.

In [2]:
# set environment variables 
%env MONGO_HOST=localhost
%env MONGO_PORT=27017
%env MONGO_USERNAME=modelci
%env MONGO_PASSWORD=modelci@2020
%env MONGO_DB=modelci
%env MONGO_AUTH_SOURCE=modelci
%env BACKEND_CORS_ORIGINS="http://localhost,http://localhost:3000,http://localhost:8080,https://localhost:3000,https://localhost:8080"
%env PROJECT_NAME=modelci
%env SECRET_KEY=2a6c03b9ca06cd8fc3cf506f0ba924cb735f15918d54758426fd7282366a5e19

env: MONGO_HOST=localhost
env: MONGO_PORT=27017
env: MONGO_USERNAME=modelci
env: MONGO_PASSWORD=modelci@2020
env: MONGO_DB=modelci
env: MONGO_AUTH_SOURCE=modelci
env: BACKEND_CORS_ORIGINS="http://localhost,http://localhost:3000,http://localhost:8080,https://localhost:3000,https://localhost:8080"
env: PROJECT_NAME=modelci
env: SECRET_KEY=2a6c03b9ca06cd8fc3cf506f0ba924cb735f15918d54758426fd7282366a5e19


Then start the modelci service by following command:

In [3]:
%%bash
modelci start

2020-11-24 15:31:25.233664: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-24 15:31:32,443 - ml-modelci Docker Container Manager - INFO - Container name=mongo-25975 stared
2020-11-24 15:31:33,523 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-8488 started.
2020-11-24 15:31:35,900 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-7678 started.
2020-11-24 15:31:37,025 - ml-modelci Docker Container Manager - INFO - gpu-metrics-exporter-67720 stared
2020-11-24 15:31:37,288 - modelci backend - INFO - Uvicorn server listening on 8000
2020-11-24 15:31:39.156824: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:     Started server process [66398]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to

## 2. Register ResNet50 Model


Firstly, we load pre-trained resnet50 model from torchvision, you can refer to <https://pytorch.org/docs/stable/torchvision/models.html> for more examples of pretrained models.

In [4]:
# load resnet50 model from torchvision
from torchvision import models
model = models.resnet50(pretrained=True)

Then we register this model into ModelHub

In [5]:
from modelci.hub.manager import register_model
from modelci.types.bo import Framework, IOShape, ModelVersion,Engine
from modelci.types.trtis_objects import ModelInputFormat
# set model input and output formats
inputs = [IOShape([-1, 3, 224, 224], dtype=float, name='INPUT__0', format=ModelInputFormat.FORMAT_NCHW)]
outputs = [IOShape([-1, 1000], dtype=float, name='probs')]
# register the model
register_model(
    model,
    dataset='imagenet',
    acc=0.76,
    task='image classification',
    inputs=inputs,
    outputs=outputs,
    architecture='ResNet50',
    framework=Framework.PYTORCH,
    version=ModelVersion('2')
)

2020-11-24 15:38:39,221 - converter - INFO - Use cached model
2020-11-24 15:38:39,224 - converter - INFO - Use cached model


## 3. Retrieve Models
By default, Converter will automatically convert registered models into optimized formats,PyTorch model can be converted to TorchScipt and ONNX formats, so we can retrieve two models from ModelHub.

In [6]:
# search for model
from modelci.hub.manager import retrieve_model
retrieved_models = retrieve_model(
        architecture_name = 'ResNet50',
        framework = Framework.PYTORCH,
        version=ModelVersion('2')
)

In [7]:
retrieved_models

We can compare detatiled information of these two models

In [8]:
retrieved_models[0].__dict__

{'_id': '5fbcb8810f85b166265ea94d',
 'name': 'ResNet50',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.TORCHSCRIPT: 2>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7fd690720990>,
 'dataset': 'imagenet',
 'acc': 0.76,
 'task': 'image classification',
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7fd690720110>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7fd690720710>],
 'weight': <modelci.types.bo.model_objects.Weight at 0x7fd690720890>,
 'profile_result': None,
 'status': <Status.UNKNOWN: 0>,
 'creator': 'sherry',
 'create_time': datetime.datetime(2020, 11, 24, 15, 38, 38, 594000)}

In [9]:
retrieved_models[1].__dict__

{'_id': '5fbcb8830f85b166265eaad7',
 'name': 'ResNet50',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.ONNX: 3>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7fd6907a3b10>,
 'dataset': 'imagenet',
 'acc': 0.76,
 'task': 'image classification',
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7fd6907f2250>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7fd6e0cc8d90>],
 'weight': <modelci.types.bo.model_objects.Weight at 0x7fd6e0cc8d10>,
 'profile_result': None,
 'status': <Status.UNKNOWN: 0>,
 'creator': 'sherry',
 'create_time': datetime.datetime(2020, 11, 24, 15, 38, 38, 594000)}

## 4. Convert models
We can convert models mannually. 

You can refer to <https://github.com/cap-ntu/ML-Model-CI/blob/master/docs/tutorial/convert.md> for more details.

In the following example, we will convert ONNX model into TensorRT format.

In [10]:
from modelci.hub.converter import TRTConverter
from modelci.hub.utils import generate_path
from modelci.types.bo import IOShape

# set model input and output formats
inputs = [IOShape([-1, 3, 224, 224], dtype=float, name='INPUT__0', format=ModelInputFormat.FORMAT_NCHW)]
outputs = [IOShape([-1, 1000], dtype=float, name='probs')]

In [11]:
# get ONNX model saved path
onnx_path = retrieved_models[1].saved_path
onnx_path

PosixPath('/home/sherry/.modelci/ResNet50/pytorch-onnx/2.onnx')

In [12]:
# set TensorRT format model save path
save_path = generate_path(
    model_name='ResNet50', framework=Framework.PYTORCH,engine=Engine.TRT,version=2
)
save_path

PosixPath('/home/sherry/.modelci/ResNet50/pytorch-trt/2')

In [13]:
from modelci.hub.converter import PyTorchConverter
TRTConverter.from_onnx(onnx_path, save_path, inputs=inputs, outputs=outputs)

Loading ONNX file from path /home/sherry/.modelci/ResNet50/pytorch-onnx/2.onnx...


True

## 5.Stop the ModelCI service

In [14]:
%%bash
modelci stop

2020-11-24 15:39:58.746586: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-24 15:40:03,246 - ml-modelci Docker Container Manager - INFO - Container name=gpu-metrics-exporter-67720 stopped.
2020-11-24 15:40:04,698 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-7678 stopped.
2020-11-24 15:40:05,294 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-8488 stopped.
2020-11-24 15:40:06,276 - ml-modelci Docker Container Manager - INFO - Container name=mongo-25975 stopped.
2020-11-24 15:40:06,493 - modelci backend - INFO - The Uvicorn server with pid=68840 stopped.


In [15]:
%%bash
# remove running docker containers
docker rm $(docker ps -a -q)

7f38a9bd32f7
45a1bcfe0b40
028f6bdd1f10
6ca5163efbfc
